In [1]:
import os
import sys
from collections import defaultdict
import matplotlib.pyplot as plt
import math
import numpy as np
from numpy import gradient
import pickle
from packages.helper import traj_speed
from packages import data_container
from packages.data_container import Data
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

In [26]:
header = ('subj_x', 'subj_y', 'subj_yaw')

info = {'p_goal':[], 'subj_id':[], 'trial_id':[], 'stimuli_onset':[], 'goal_angle':[], 'w_goal': 0.1, 'w_obst': 0.1,
        'ps_trial':[], 'ps_subj':[], 'time_stamp':[], 'p_subj':[], 'v_subj':[], 'a_subj':[]}
data = Data(Hz=30, header=header, info=info)

data_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'F&W2003_Exp1a_rawData'))
ii = 0
goal_side = 0 # +1 means on the right side, -1 means on the left side
for data_file in os.listdir(data_dir):
    i_subj = int(data_file[-1])
    data_file_path = os.path.join(data_dir, data_file)
    i_trial = 0
    with open(data_file_path, 'r') as f:
        trials = f.read().split('\n')
    head = 4
    length = int(trials[head])
    while True:
        # import data
        subj = [line.split('\t') for line in trials[head + 1: head + 1 + length]]
        subj = np.array([[float(vals[0])/100, float(vals[1])/100, float(vals[2])] for vals in subj])
        data.add_traj(subj)
        time_stamp = np.linspace(1/data.Hz, len(subj)/data.Hz, len(subj))
        p_subj = Data.filter(subj[:,0:2], time_stamp, data.Hz, 4, 0.6)
        v_subj = gradient(p_subj, axis=0) * data.Hz
        a_subj = gradient(v_subj, axis=0) * data.Hz
        # Use the average speed from the -3 to -2 second as the trial preferred speed
        # because the goal is close and subjects had to slow down early
        ps = np.mean(traj_speed(p_subj[-3*data.Hz : -2*data.Hz], data.Hz))
        # import info
        for i in range(len(subj)):
            if subj[i][1] >= 1:
                stimuli_onset = i
                break
        if trials[head-1].split(' ')[0] == '0':
            cond = trials[head-2].split(' ')
            goal_side = float(cond[2])
            goal_angle = goal_side * float(cond[3]) * 2 * math.pi / 360
            goal_dist = float(cond[4]) / 100
        p_goal = [(goal_dist * math.sin(goal_angle), goal_dist * math.cos(goal_angle)+1)] * length
        p_goal = np.array(p_goal)
        angle = round(goal_angle * 360 / 2 / math.pi)
        data.add_info({'p_goal':p_goal, 'subj_id':i_subj, 'trial_id':i_trial, 'stimuli_onset':stimuli_onset,
                       'goal_angle':angle, 'ps_trial':ps, 'ps_subj':0, 'time_stamp':time_stamp,
                       'p_subj':p_subj, 'v_subj':v_subj, 'a_subj':a_subj})

        if head + length + 3 >= len(trials):
            break

        if len(trials[head+length+3].split(' ')) == 1:
            head = head+length+3
        else:
            head = head+length+4
        length = int(trials[head])
        ii += 1
        i_trial += 1

# Setting subject preferred speed to every trial
ps_subj = defaultdict(list)
for i in range(len(data.trajs)):
    subj_id = data.info['subj_id'][i]
    ps_subj[subj_id].append(data.info['ps_trial'][i])
for i in range(len(data.trajs)):
    subj_id = data.info['subj_id'][i]
    data.info['ps_subj'][i] = np.mean(ps_subj[subj_id])

outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Fajen_steer_exp1a_data.pickle'))
with open(outfile, 'wb') as file:   
    pickle.dump(data, file, pickle.HIGHEST_PROTOCOL) 

In [28]:
# Check imported data
vals = []
%matplotlib qt
for i in range(len(data.trajs)):
    val = data.info['ps_trial'][i]
    vals.append(val)
    p_goal = np.array(data.info['p_goal'][i])
    p_subj = np.array(data.get_traj(i))
    onset = data.info['stimuli_onset'][i]
    i_subj = data.info['subj_id'][i]
    angle = data.info['goal_angle'][i]
    plt.plot(vals)
#     if i_subj == 0:
#         plt.plot(traj_speed(data.info['p_subj'][i], data.Hz))
#     if angle == -25:
#         plt.plot(data.info['p_subj'][i][:,0], data.info['p_subj'][i][:,1])
        

